In [5]:
import random
import nltk, re, pprint
from nltk import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import matplotlib.pyplot as plt
import copy
import numpy as np
import seaborn as sns
import itertools
import pickle
import math
import networkx as nx
import community
import pickle 
from collections import Counter
from wordcloud import WordCloud, ImageColorGenerator
from operator import itemgetter
from fa2 import ForceAtlas2
sns.set()
%matplotlib inline

In [6]:
df = pd.read_pickle("./movie_scripts.pkl")
df.head()

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
0,1,1,Albus Dumbledore,I should have known that you would be here...P...,A neighbourhood on a street called Privet Driv...,1
1,1,2,Mcgonagall,"Good evening, Professor Dumbledore. Are the ru...","The cat meows, sniffs out and the camera pans ...",1
2,1,3,Albus Dumbledore,"I'm afraid so, Professor. The good, and the bad.","The cat meows, sniffs out and the camera pans ...",1
3,1,4,Mcgonagall,And the boy?,"The cat meows, sniffs out and the camera pans ...",1
4,1,5,Albus Dumbledore,Hagrid is bringing him.,"The cat meows, sniffs out and the camera pans ...",1


In [7]:
# All the movie titles in correct order
move_titles = ["Harry Potter and the Philosopher's Stone","Harry Potter and the Chamber of Secrets",
              "Harry Potter and the Prisoner of Azkaban","Harry Potter and the Goblet of Fire",
              "Harry Potter and the Order of the Phoenix","Harry Potter and the Half-Blood Prince",
              "Harry Potter and the Deathly Hallows – Part 1","Harry Potter and the Deathly Hallows – Part 2"]
move_titles_short = [move_titles[i].split("Harry Potter and the ", 1)[1] for i in range(len(move_titles))]

In [8]:
# Add new column to dataframe that include the tokenized sentences
# Function to calculate the sentiment of tokenized list
def tokenize(text):
    #toxenize the text 
    tokens = nltk.word_tokenize(text)
    words = [w.lower() for w in tokens if w.isalpha()]
    lem_words = [wnl.lemmatize(w) for w in words if w not in stopwords]
    return(lem_words)

stopwords = nltk.corpus.stopwords.words('english')
wnl = nltk.WordNetLemmatizer()

In [9]:
# Lets combine all sentences of characters into a dictionary

char_sentences = dict()
for character in df.character.unique():
    text = tokenize(df.sentence[df.character == character].str.cat(sep=' '))
    char_sentences[character] = text
    
# Lets combine all sentences for each movie into a dictionary
movie_sentences = dict()
for movie in df.movie_nr.unique():
    text = tokenize(df.sentence[df.movie_nr == movie].str.cat(sep=' '))
    movie_sentences[movie] = text
    
# maybe have dictionary that has the key movie, that has the key of character and and the value as sentence

In [10]:
nltk.FreqDist(movie_sentences[1])

FreqDist({'harry': 161, 'go': 59, 'see': 53, 'come': 50, 'ron': 50, 'one': 49, 'oh': 48, 'know': 47, 'hagrid': 44, 'potter': 39, ...})

Þinn kóði fyrir neðan (ekki búinn að fitta rétt) :D 

In [ ]:
# Collect all the top 10 most common words in each book into a set
most_common_words = []

for i in range(len(books)):
    terms = [term for (term, freq) in books[i]['fdist_norm'].most_common(10)]
    most_common_words.extend(terms)

word_set = list(set(most_common_words))

# Collect the total relative frequency of each word so that the order can be defined
word_set_freq = []
for word in word_set:
    word_freq = 0
    for i in range(len(books)):
        word_freq += books[i]['fdist_norm'][word]
    word_set_freq.append(word_freq)

# create a list of the correctly sorted elements
word_set_sorted = list(np.array(word_set)[np.argsort(word_set_freq).astype(int)])

In [ ]:
fig, axes = plt.subplots(1, 7, sharey=True, sharex=True, figsize=(20,10), gridspec_kw={'wspace':0.01, 'hspace':0.01}, squeeze=True)

palette = itertools.cycle(sns.color_palette())

for i in range(7):

    height = []

    for word in word_set_sorted:
        height.append(books[i]['fdist_norm'][word])

    # Create the list of bars for the plot
    bars = word_set_sorted

    y_pos = np.arange(len(bars))

    # Create horizontal bars
    axes[i].barh(y_pos, height, color=next(palette))
    
axes[i]
fig.set_tight_layout(True)

# Create names on the y-axis
plt.yticks(np.arange(len(bars)), word_set_sorted);
fig.suptitle('highest normalized frequency of words in all the books');
# Set the rotation and horizontal alignment of the x tick labels
#plt.setp(axes[1][0].get_xticklabels(), rotation=45, ha="right");
#plt.setp(axes[1][1].get_xticklabels(), rotation=45, ha="right");